<font color='green'> 
**Youtube - Aladdin Persson Kanalı - TensorFlow 2.0 Beginner Tutorials serisi**
    
TensorFlow Tutorial 16 - Custom Training Loops - Aladdin Persson anlattı.
</font>

**Video**: [TensorFlow Tutorial 16 - Custom Training Loops](https://www.youtube.com/watch?v=_u7AVsxANes&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=16)

### İçindekiler

**Loading, Preprocessing Dataset and Creating Model**

**Creating Training Loop (model.fit)**

**Creating Test Loop (model.evaluate)**



### <font color="blue"> Giriş</font>

Sıfırdan nasıl training loop ve test loop yapacağımızı işleyeceğiz. Yani artık model.fit() ve model.evaluate() kullanmayacağız her şeyi kendimiz sıfırdan yazacağız. 

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_datasets as tfds

### 1. Loading, Preprocessing Dataset and Creating Model

In [10]:
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [11]:
def normalize_img(image,label):
  return tf.cast(image, tf.float32) / 255.0, label

In [13]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

In [14]:
 model = keras.Sequential(
     [
      keras.Input((28,28,1)),
      layers.Conv2D(32, 3, activation="relu"),
      layers.Flatten(),
      layers.Dense(10, activation = "softmax")
     ]
 )

### 2. Creating Training Loop (model.fit)

In [15]:
num_epochs = 5
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
acc_metric = keras.metrics.SparseCategoricalAccuracy()

model.fit() yerine yazıyoruz bunu.

In [17]:
for epoch in range(num_epochs):
  print(f"\nStart of Training Epoch {epoch}")

  for batch_idx, (x_batch, y_batch) in enumerate (ds_train): # her epochta her batchimiz için bu işlemi yapacağız.
    with tf.GradientTape() as tape:    # forward propagationdaki tüm operasyonları kaydetmek için
      y_pred = model(x_batch, training=True)
      loss = loss_fn(y_batch, y_pred) # yukarıda belirttiğimiz fonksiyon
      
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    acc_metric.update_state(y_batch, y_pred) 

  train_acc = acc_metric.result()
  print(f"Accuracy over epoch {train_acc}")
  acc_metric.reset_states() # bir sonraki epoch için sıfırlıyoruz accuracyi.



Start of Training Epoch 0
Accuracy over epoch 0.9089999794960022

Start of Training Epoch 1
Accuracy over epoch 0.913433313369751

Start of Training Epoch 2
Accuracy over epoch 0.9159166812896729

Start of Training Epoch 3
Accuracy over epoch 0.9189333319664001

Start of Training Epoch 4
Accuracy over epoch 0.9214000105857849


### 3. Creating Test Loop (model.evaluate)

model.evaluate() yerine bunu yazdık.

In [19]:
# Birden fazla epoch için çalıştırmamız gerekmiyor. Data setini bir kere çalıştırmamız yeterli.

for x, y in ds_test: 
  y_pred = model(x, training=False)
  acc_metric.update_state(y, y_pred)

test_acc = acc_metric.result()
print(f"Accuracy over Test Set: {test_acc}")

Accuracy over Test Set: 0.9085999727249146


* `for batch_idx, (x_batch, y_test) in enumerate(ds_test):` burada batch_idx kullanmamıza ve dolayısıyla enumerate kullanmamıza gerek yok ama bazen ihtiyaç olabiliyor. Ben bunun yerine `for x, y in ds_test:` yazdım.

* ` with tf.GradientTape() as tape:` buna ihtiyacımız yok çünkü gradientleri toplamayacağız.